In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [45]:
df1 = pd.read_csv('data/turnstile_160507.txt')
df2 = pd.read_csv('data/turnstile_160514.txt')
df3 = pd.read_csv('data/turnstile_160521.txt')
df4 = pd.read_csv('data/turnstile_160528.txt')
frames = [df1, df2, df3, df4]
df = pd.concat(frames)

In [43]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,diffEntries,diffExits
0,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,00:00:00,REGULAR,5672118,1920707,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,04:00:00,REGULAR,5672183,1920719,65.0,12.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,08:00:00,REGULAR,5672214,1920761,31.0,42.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,12:00:00,REGULAR,5672330,1920867,116.0,106.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,05/21/2016,16:00:00,REGULAR,5672640,1920936,310.0,69.0


In [48]:
df = df.sort(['C/A','UNIT','SCP','DATE','TIME'])

/Users/connor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [62]:
df['diffEntries'] = df.ENTRIES.diff()


In [65]:
# df.columns = (u'C/A', u'UNIT', u'SCP', u'STATION', u'LINENAME', u'DIVISION', u'DATE',
#        u'TIME', u'DESC', u'ENTRIES',
#        u'EXITS',
#        u'diffEntries')

df['diffExits'] = df.EXITS.diff()

In [66]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,diffEntries,diffExits
0,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,00:00:00,REGULAR,5639941,1909983,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,04:00:00,REGULAR,5639991,1909993,50.0,10.0
2,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,08:00:00,REGULAR,5640014,1910024,23.0,31.0
3,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,12:00:00,REGULAR,5640158,1910134,144.0,110.0
4,A002,R051,02-00-00,59 ST,NQR456,BMT,04/30/2016,16:00:00,REGULAR,5640454,1910197,296.0,63.0


In [74]:
df['machine_change'] = (df.SCP != df.SCP.shift())

In [84]:
dfrei = df.reset_index(drop=True)

In [117]:
df = dfrei

In [118]:
df.diffEntries[df.index[df.machine_change==True]] = 0

/Users/connor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [119]:
df.diffExits[df.index[df.machine_change==True]] = 0

/Users/connor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [154]:
df['counterReset'] = ((df.diffEntries > 6000) | (df.diffEntries < 0) | (df.diffExits > 6000) | (df.diffExits < 0)) 

In [155]:
df.diffExits[df.index[df.counterReset==True]] = 0

/Users/connor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [156]:
df.diffEntries[df.index[df.counterReset==True]] = 0


/Users/connor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [157]:
df['TotalVol'] = df.diffEntries+df.diffExits

In [165]:
top50 = df.groupby(['STATION']).sum()

In [168]:
top50 = top50.sort('TotalVol',ascending=False)

/Users/connor/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [169]:
top50.head()

,ENTRIES,EXITS,diffEntries,diffExits,machine_change,counterReset,TotalVol
STATION,,,,,,,
34 ST-PENN STA,488881015020,287627882120,4206635.0,3584248.0,100.0,0.0,7790883.0
GRD CNTRL-42 ST,744724963097,146796271108,3584257.0,3203623.0,61.0,0.0,6787880.0
34 ST-HERALD SQ,563482244287,596750042788,3026299.0,2719583.0,54.0,0.0,5745882.0
23 ST,1111704579992,1034681559184,2922280.0,2239758.0,71.0,0.0,5162038.0
86 ST,394240269476,430100768929,2819901.0,2316859.0,52.0,0.0,5136760.0


In [170]:
top50.to_csv('volumeByStation')